In [33]:
fertilizer_json = {
    "Kalkamonsalpeter": {
        "Zusammensetzung": 0.27,
        "Emissionsfaktor": 7.101
    },
    "Harnstoff": {
        "Zusammensetzung": 0.46,
        "Emissionsfaktor": 12.4
    },
    "ASS": {
        "Zusammensetzung": 0.26,
        "Emissionsfaktor": 11.438
    },
    "Diammonphosphat": {
        "Zusammensetzung": {
            "N": 0.18,
            "P2O5": 0.46
        },
        "Emissionsfaktor": 11.958
    },
    "Gülle (Organisch)": {
        "Zusammensetzung": 1.0,
        "Emissionsfaktor": 4.97
    }
}

In [34]:
# Cell 1: Import Libraries
import geopandas as gpd
import pandas as pd
import glob
import os
from shapely.geometry import MultiPoint

# Set SHAPE_RESTORE_SHX environment variable
os.environ["SHAPE_RESTORE_SHX"] = "YES"

# Cell 2: Define Root Folder and Initialize List
root_folder = "data/Erntejahr 2022 (Weizen, Düngung)"

# Initialize an empty list to store shapefile paths
shapefile_list = []

# Traverse the directory structure
for subdir, _, _ in os.walk(root_folder):
    # Look for .shp files in the 'doc' folder of each subdirectory
    shapefiles = glob.glob(os.path.join(subdir, "doc", "*.shp"))
    shapefile_list.extend(shapefiles)


# Cell 3: Read Shapefiles and Combine into a Single GeoDataFrame
gdfs = []
for shapefile_path in shapefile_list:
    gdf = gpd.read_file(shapefile_path)
    # Append the GeoDataFrame to a list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Fill missing values with NA
combined_gdf = combined_gdf.fillna('NA')

# Cell 4: Display Output and Columns
print("All shapefiles combined successfully!")
# Uncomment the lines below if you want to save the combined shapefile
# Cell 5: Save Combined GeoDataFrame as a Shapefile
# output_shapefile_path = os.path.join(root_folder, "combined_shapefile.shp")
# combined_gdf.to_file(output_shapefile_path)


All shapefiles combined successfully!


In [ ]:
combined_gdf['Product'] = combined_gdf['Product'].apply(lambda x: x.split()[0])

# Cell 6: Calculate Emissions for Each Product
def calculate_emissions(row):
    product = row["Product"]
    appliedRate = row["AppliedRate"]
    if product in fertilizer_json:
        if fertilizer_json[product]["Zusammensetzung"]:
            if isinstance(fertilizer_json[product]["Zusammensetzung"], dict):
                composition = fertilizer_json[product]["Zusammensetzung"]["N"]
            else:
                composition = fertilizer_json[product]["Zusammensetzung"]
            emissions_factor = fertilizer_json[product]["Emissionsfaktor"]
            emissions = composition * appliedRate * emissions_factor
        else:
            emissions = appliedRate * fertilizer_json[product]["Emissionsfaktor"]
    else:
        emissions = 0
    return emissions

combined_gdf["Emissions"] = combined_gdf.apply(calculate_emissions, axis=1)
combined_gdf

In [36]:
average_emissions = combined_gdf['Emissions'].mean()
print(average_emissions)

244.63274829531895


In [37]:
combined_gdf_grouped = combined_gdf.groupby("Product").agg({"Emissions": "sum"})
print(combined_gdf_grouped)

                     Emissions
Product                       
ASS               3.982442e+06
Gärsubstrat       0.000000e+00
Kalkamonsalpeter  2.123308e+07
